In [1]:
%run "SparkApp.ipynb"

24/12/26 16:02:34 WARN Utils: Your hostname, Ala resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/12/26 16:02:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ala/.ivy2/cache
The jars for the packages stored in: /home/ala/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c40237ec-1768-49e8-a964-6ceef1790b65;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.0 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution

In [2]:
spark

In [9]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType, TimestampType

# Définition du schéma PySpark pour les données provenant de Kafka
schema = StructType([
    StructField("key", StructType([
        StructField("value", StringType(), True),
        StructField("topic", StringType(), True),
        StructField("partition", LongType(), True),
        StructField("offset", LongType(), True),
        StructField("timestamp", TimestampType(), True),
        StructField("timestampType", LongType(), True)
    ]), True),
    StructField("data", StructType([
        StructField("DOO", StructType([
            StructField("desi", StringType(), True),
            StructField("dir", StringType(), True),
            StructField("oper", LongType(), True),
            StructField("veh", LongType(), True),
            StructField("tst", StringType(), True),
            StructField("tsi", LongType(), True),
            StructField("spd", DoubleType(), True),
            StructField("hdg", LongType(), True),
            StructField("lat", DoubleType(), True),
            StructField("long", DoubleType(), True),
            StructField("acc", DoubleType(), True),
            StructField("dl", LongType(), True),
            StructField("odo", LongType(), True),
            StructField("drst", LongType(), True),
            StructField("oday", StringType(), True),
            StructField("jrn", LongType(), True),
            StructField("line", LongType(), True),
            StructField("start", StringType(), True),
            StructField("loc", StringType(), True),
            StructField("stop", LongType(), True),
            StructField("route", StringType(), True),
            StructField("occu", LongType(), True)
        ]), True)
    ]), True)
])



In [4]:
kafka_df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "vip") \
    .option("startingOffsets","earliest") \
    .load()

In [5]:
kafka_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [6]:
from pyspark.sql.functions import expr

kafka_json_df = kafka_df.withColumn("value", expr("cast(value as string)"))

In [7]:
from pyspark.sql.functions import from_json,col

streaming_df = kafka_json_df.selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), schema).alias("data"))

In [8]:
streaming_df.printSchema()

root
 |-- data: struct (nullable = true)
 |    |-- DOO: struct (nullable = true)
 |    |    |-- desi: string (nullable = true)
 |    |    |-- dir: string (nullable = true)
 |    |    |-- oper: long (nullable = true)
 |    |    |-- veh: long (nullable = true)
 |    |    |-- tst: string (nullable = true)
 |    |    |-- tsi: long (nullable = true)
 |    |    |-- spd: double (nullable = true)
 |    |    |-- hdg: long (nullable = true)
 |    |    |-- lat: double (nullable = true)
 |    |    |-- long: double (nullable = true)
 |    |    |-- acc: double (nullable = true)
 |    |    |-- dl: long (nullable = true)
 |    |    |-- odo: long (nullable = true)
 |    |    |-- drst: long (nullable = true)
 |    |    |-- oday: string (nullable = true)
 |    |    |-- jrn: long (nullable = true)
 |    |    |-- line: long (nullable = true)
 |    |    |-- start: string (nullable = true)
 |    |    |-- loc: string (nullable = true)
 |    |    |-- stop: long (nullable = true)
 |    |    |-- route: string (n

In [11]:
from pyspark.sql import functions as F

# Aplatir et sélectionner les colonnes de DOO
flattened_df = streaming_df.select(
    F.col("data.DOO.desi").alias("desi"),
    F.col("data.DOO.dir").alias("dir"),
    F.col("data.DOO.oper").alias("oper"),
    F.col("data.DOO.veh").alias("veh"),
    F.col("data.DOO.tst").alias("tst"),
    F.col("data.DOO.tsi").alias("tsi"),
    F.col("data.DOO.spd").alias("spd"),
    F.col("data.DOO.hdg").alias("hdg"),
    F.col("data.DOO.lat").alias("lat"),
    F.col("data.DOO.long").alias("long"),
    F.col("data.DOO.acc").alias("acc"),
    F.col("data.DOO.dl").alias("dl"),
    F.col("data.DOO.odo").alias("odo"),
    F.col("data.DOO.drst").alias("drst"),
    F.col("data.DOO.oday").alias("oday"),
    F.col("data.DOO.jrn").alias("jrn"),
    F.col("data.DOO.line").alias("line"),
    F.col("data.DOO.start").alias("start"),
    F.col("data.DOO.loc").alias("loc"),
    F.col("data.DOO.stop").alias("stop"),
    F.col("data.DOO.route").alias("route"),
    F.col("data.DOO.occu").alias("occu")
)

In [12]:
flattened_df.printSchema()

root
 |-- desi: string (nullable = true)
 |-- dir: string (nullable = true)
 |-- oper: long (nullable = true)
 |-- veh: long (nullable = true)
 |-- tst: string (nullable = true)
 |-- tsi: long (nullable = true)
 |-- spd: double (nullable = true)
 |-- hdg: long (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- acc: double (nullable = true)
 |-- dl: long (nullable = true)
 |-- odo: long (nullable = true)
 |-- drst: long (nullable = true)
 |-- oday: string (nullable = true)
 |-- jrn: long (nullable = true)
 |-- line: long (nullable = true)
 |-- start: string (nullable = true)
 |-- loc: string (nullable = true)
 |-- stop: long (nullable = true)
 |-- route: string (nullable = true)
 |-- occu: long (nullable = true)

